In [1]:
 # Change this to a directory that's not the NumPy source directory
#load libraries
import json
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import logging
import csv

from bs4 import BeautifulSoup
import requests
from datetime import datetime
import locale
from concurrent.futures import ThreadPoolExecutor
from lxml import html
from tqdm import tqdm
import time
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from wordcloud import WordCloud  # Import wordcloud package

In [4]:
#set paths
download_folder = './downloads'
gecko_driver_path = '/Users/amberwalker/Intro_text_nlp/ps1/geckodriver'
profile_path =  '/Users/amberwalker/Library/Application Support/Firefox/Profiles/9eq5v9xc.default-release-1'
link = 'https://www.fool.com/quote/nyse/rcl/'
file_path_out ='output.csv'
file_path = '/Users/amberwalker/Intro_text_nlp/final_project/final_texts.csv'

In [3]:
def set_firefox_preferences(download_folder, profile_path, enable_download=False):
    """
    Configures Firefox preferences for Selenium WebDriver.
    """
    # Use the provided profile path
    profile = webdriver.FirefoxProfile(profile_path)

    # Set preferences
    profile.set_preference("browser.download.dir", download_folder)
    profile.set_preference("browser.download.folderList", 2)
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk",
                           "application/msword,application/rtf,application/csv,text/csv,image/png,image/jpeg,application/pdf,text/html,text/plain,application/octet-stream")

    # Automatically download PDFs
    if enable_download:
        profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf,application/x-pdf")
        profile.set_preference("pdfjs.disabled", True)

    options = Options()
    options.profile = profile
    return options

def start_firefox(link, download_folder, gecko_driver_path, profile_path, enable_download=True):
    """
    Initializes a Firefox WebDriver session.
    """
    # Explicitly specify the path to the Firefox executable
    options = Options()
    options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"

    # Ensure the download folder exists
    os.makedirs(download_folder, exist_ok=True)

    # Set Firefox preferences with the given profile
    options = set_firefox_preferences(download_folder, profile_path, enable_download)

    # Set up the service with the path to the geckodriver
    service = Service(executable_path=gecko_driver_path)
    browser = webdriver.Firefox(options=options, service=service)
    browser.get(link)
    time.sleep(5)  # Adjust sleep time as needed
    return browser

def perform_click_action(browser, xpath, click_type):
    """
    Tries to click on an element. If obscured, waits and retries.
    """
    attempt_count = 0
    while attempt_count < 15:
        if try_click_element(browser, xpath, click_type):
            return
        time.sleep(1)
        attempt_count += 1
    # Optional: Add warning sound or logging here if needed

def try_click_element(browser, xpath, click_type):
    """
    Attempts to click an element based on its type.
    """
    try:
        element = {
            "xpath": lambda: browser.find_element_by_xpath(xpath),
            "id": lambda: browser.find_element_by_id(xpath),
            "css": lambda: browser.find_element_by_css_selector(xpath),
            "class": lambda: browser.find_element_by_class_name(xpath),
            "link": lambda: browser.find_element_by_link_text(xpath)
        }.get(click_type, lambda: None)()

        if element:
            element.click()
            return True
    except (ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException) as e:
        print(f"Error encountered: {e}")
    return False

# Function that collects the link to then later open them 
def collect_transcript_links(browser):
    links = []
    try:
        # Initially collect available links before clicking "View More"
        transcript_elements = browser.find_elements(By.CSS_SELECTOR, 'a[data-track-category="quotepage_transcripts"]')
        links.extend([element.get_attribute('href') for element in transcript_elements])
        
        while True:
            # Try to click "View More" button to load more transcripts
            #view_more_button = browser.find_element(By.XPATH, "//span[contains(text(), 'View More RCL Earnings Transcripts')]")
            view_more_button = browser.find_element(By.XPATH, "//span[contains(text(), 'View More') and contains(text(), 'Earnings Transcripts')]")

            browser.execute_script("arguments[0].click();", view_more_button)
            time.sleep(2)  # Adjust based on actual page load time
            
            # Update the list of transcript elements after loading more
            new_transcript_elements = browser.find_elements(By.CSS_SELECTOR, 'a[data-track-category="quotepage_transcripts"]')
            new_links = [element.get_attribute('href') for element in new_transcript_elements]
            
            # Check if new links were added after clicking "View More"
            if len(new_links) > len(links):
                print(f"Collected {len(new_links) - len(links)} new links. Total collected: {len(new_links)}")
                links = new_links  # Update the links list with the new set of links
            else:
                # No new links were added indicating all links have been loaded or "View More" is not working as expected
                print("No new links collected. Assuming all links have been collected.")
                break
    
    except Exception as e:
        # Handle cases where "View More" button is not found or other errors occur
        print("No more 'View More' button found or an error occurred:", str(e))
    
    return links

#function to scrape ALL text from each link (webpage)
def scrape_all_text(url):
    browser = webdriver.Chrome()
    browser.get(url)
    
    try:
        # Wait for the page to load
        WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        
        # Scrape all text
        all_text = browser.find_element(By.TAG_NAME, "body").text
        return all_text
    finally:
        browser.quit()
        
def extract_executive_speeches_from_text(text, ceo_info, cfo_info):
    speeches = {'CEO': [], 'CFO': [], 'Title': '', 'Date': ''}
    current_executive = None
    speech_lines = []
    lines = text.split('\n')
    
    # Search for title and date in the text
    for line in lines:
        title_pattern = re.compile(r'(.*?)\s+\((.*?)\)\s+(Q[1-4]\s+\d{4})\s+Earnings Call Transcript')
        date_pattern = re.compile(r'period ending (\w+ \d{1,2}, \d{4})')
        title_match = title_pattern.search(line)
        date_match = date_pattern.search(line)
        if title_match:
            speeches['Title'] = title_match.group(0)  # The entire match
        if date_match:
            speeches['Date'] = date_match.group(1)

    for line in lines:
        if any(name in line for name in ceo_info['names']) or any(keyword in line for keyword in ceo_info['keywords']):
            if speech_lines and current_executive:
                speeches[current_executive].append(' '.join(speech_lines))
                speech_lines = []
            current_executive = 'CEO'
        elif any(name in line for name in cfo_info['names']) or any(keyword in line for keyword in cfo_info['keywords']):
            if speech_lines and current_executive:
                speeches[current_executive].append(' '.join(speech_lines))
                speech_lines = []
            current_executive = 'CFO'

        if current_executive:
            speech_lines.append(line.strip())
            
            # Heuristic to detect potential end of speech
            if "--" in line:
                speeches[current_executive].append(' '.join(speech_lines))
                speech_lines = []
                current_executive = None

    # Ensure the last captured speech is saved
    if speech_lines and current_executive:
        speeches[current_executive].append(' '.join(speech_lines))

    return speeches

In [14]:
# Step 1: Collect Transcript Links
companies = [
    {'symbol': 'RCL', 'url': 'https://www.fool.com/quote/nyse/rcl/'},
    {'symbol': 'CCL', 'url': 'https://www.fool.com/quote/nyse/ccl/'},
    {'symbol': 'NCLH', 'url': 'https://www.fool.com/quote/nyse/nclh/'}
]

transcript_link_by_company = {} #dictionary to store company info

for company in companies:
    # Extract company-specific information
    symbol = company['symbol']
    company_url = company['url']
    
    # Start up the browser for each company's URL
    browser = start_firefox(company_url, download_folder, gecko_driver_path, profile_path)
    
    # Collect transcript links for the current company
    company_transcript_links = collect_transcript_links(browser)
    
    # Store the links in the dictionary under the current company's symbol
    transcript_link_by_company[symbol] = company_transcript_links
    
    browser.quit()

Collected 4 new links. Total collected: 8
Collected 4 new links. Total collected: 12
Collected 4 new links. Total collected: 16
Collected 4 new links. Total collected: 20
Collected 2 new links. Total collected: 22
No new links collected. Assuming all links have been collected.
Collected 4 new links. Total collected: 8
Collected 4 new links. Total collected: 12
Collected 4 new links. Total collected: 16
Collected 4 new links. Total collected: 20
No new links collected. Assuming all links have been collected.
Collected 4 new links. Total collected: 8
Collected 4 new links. Total collected: 12
Collected 4 new links. Total collected: 16
Collected 3 new links. Total collected: 19
No new links collected. Assuming all links have been collected.


In [22]:
# Dictionary to store each company's DataFrame
company_dataframes = {}

for symbol, links in transcript_link_by_company.items():
    company_texts = []

    for link in links:
        scraped_text = scrape_all_text(link)
        company_texts.append({
            'Link': link,
            'Text': scraped_text
        })
    
    # Create a DataFrame for the current company's texts
    df = pd.DataFrame(company_texts)
    
    # Store the DataFrame in the dictionary using the company symbol as the key
    company_dataframes[symbol] = df

In [25]:
#looking for keywords and titles to identify whether it's CEO or CFO
ceo_info = {
    'names': ["ArnoldDonald", "Frank J. Del Rio", "Josh Weinstein", "Arnold Donald", "Frank Del Rio", "Michael Bayley", "Harry Sommer"],
    'keywords': ["President and Chief Executive Officer, Norwegian Cruise Line Holdings", "Chief Executive Officer", "CEO", "President", "President and Chief Executive Officer", "President & Chief Executive Officer"]
}

cfo_info = {
    'names': ["Mark A. Kempa", "Mark Kempa", "Jason Liberty", "Naftali Holtz", "David Bernstein"],
    'keywords': ["Executive Vice President & Chief Financial Officer", "Executive Vice President, Chief Financial Officer", "Chief Financial Officer", "CFO", "Executive Vice President, Chief Financial Officer", "Chief Financial Officer and Chief Accounting Officer", "Executive Vice President and Chief Financial Officer"]
}

#loop to iterate through DataFrame rows
results = []

for index, row in df.iterrows():
    url = row['Link']
    text = row['Text']
    
    speeches_info = extract_executive_speeches_from_text(text, ceo_info, cfo_info)
    
    title = speeches_info.get('Title', 'Unknown Title')
    date = speeches_info.get('Date', 'Unknown Date')
    
    for executive in ['CEO', 'CFO']:
        for speech in speeches_info.get(executive, []):
            results.append({
                'Link': link,
                'Title': title,
                'Date': date,
                'Executive': executive,
                'Speech': speech
            })

# Convert the results list of dictionaries into a DataFrame
final_texts = pd.DataFrame(results)

# Display the DataFrame
print(final_texts.head())

# Optionally, save the DataFrame to a CSV file
final_texts.to_csv('final_texts.csv', index=False)

                                                Link  \
0  https://www.fool.com/earnings/call-transcripts...   
1  https://www.fool.com/earnings/call-transcripts...   
2  https://www.fool.com/earnings/call-transcripts...   
3  https://www.fool.com/earnings/call-transcripts...   
4  https://www.fool.com/earnings/call-transcripts...   

                                               Title               Date  \
0  Norwegian Cruise Line (NCLH) Q4 2023 Earnings ...  December 31, 2023   
1  Norwegian Cruise Line (NCLH) Q4 2023 Earnings ...  December 31, 2023   
2  Norwegian Cruise Line (NCLH) Q4 2023 Earnings ...  December 31, 2023   
3  Norwegian Cruise Line (NCLH) Q4 2023 Earnings ...  December 31, 2023   
4  Norwegian Cruise Line (NCLH) Q4 2023 Earnings ...  December 31, 2023   

  Executive                                             Speech  
0       CEO  Thank you, Donna, and good morning, everyone. ...  
1       CEO  With that, I'd like to call -- turn the call o...  
2       CEO  Harr

In [7]:
final_df = pd.read_csv(file_path)

print(final_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2631 entries, 0 to 2630
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Link       2631 non-null   object
 1   Title      2488 non-null   object
 2   Date       2631 non-null   object
 3   Executive  2631 non-null   object
 4   Speech     2631 non-null   object
dtypes: object(5)
memory usage: 102.9+ KB
None


In [9]:
import pandas as pd

# Assuming 'final_texts' is your DataFrame
# Example keywords for demonstration
cfo_keywords = [
    "Executive Vice President & Chief Financial Officer",
    "Executive Vice President, Chief Financial Officer",
    "Chief Financial Officer",
    "CFO",
    "Executive Vice President, Chief Financial Officer",
    "Chief Financial Officer and Chief Accounting Officer",
    "Executive Vice President and Chief Financial Officer"
]

# Function to check if the speech starts with any CFO keyword
def starts_with_cfo_keyword(speech, keywords):
    for keyword in keywords:
        if speech.startswith(keyword):
            return True
    return False

# Initialize a list to keep track of rows where changes are made
changed_rows = []

# Iterate through the DataFrame and update the 'Executive' column
for index, row in final_df.iterrows():
    if row['Executive'] == 'CEO' and starts_with_cfo_keyword(row['Speech'], cfo_keywords):
        final_df.at[index, 'Executive'] = 'CFO'
        changed_rows.append(index)  # Add the index of the changed row to the list

# Report changes
if changed_rows:
    print(f"Changed the 'Executive' column for {len(changed_rows)} rows to 'CFO'.")
    for index in changed_rows:
        print(f"Row {index} was changed.")
else:
    print("No changes were made.")


No changes were made.
